# Лабораторная работа №3: Обнаружение границ и сегментация изображений

**Цель**: Освоить классические методы обнаружения границ и сегментации объектов на изображениях.

**Задачи**:
1. Применить детекторы границ (Sobel, Canny).
2. Выполнить пороговую сегментацию (глобальная, адаптивная, Otsu).
3. Реализовать сегментацию методом водораздела (watershed).

**Пример применения**: выделение деталей на конвейере СТО.

In [ ]:
# !pip install opencv-python numpy matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Загрузка изображения

In [ ]:
img = cv2.imread('parts.jpg')  # <-- укажите путь к изображению с деталями
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## 2. Обнаружение границ

In [ ]:
# ЗАДАНИЕ: примените Sobel и Canny
sobelx = None  # <-- cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobely = None  # <-- cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
sobel_combined = None  # <-- np.sqrt(sobelx**2 + sobely**2)

canny_edges = None  # <-- cv2.Canny(gray, threshold1=100, threshold2=200)

## 3. Пороговая сегментация

In [ ]:
# Глобальный порог
_, thresh_global = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Адаптивный порог
thresh_adaptive = None  # <-- cv2.adaptiveThreshold(...)

# Otsu
_, thresh_otsu = None  # <-- cv2.threshold(..., cv2.THRESH_OTSU)

## 4. Сегментация методом водораздела

In [ ]:
# Подготовка маркеров
ret, sure_fg = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# ЗАДАНИЕ: выполните морфологическое расширение для фона
kernel = np.ones((3,3), np.uint8)
sure_bg = None  # <-- cv2.dilate(gray, kernel, iterations=3)

# Получение неопределённой области
sure_fg = cv2.erode(gray, kernel, iterations=3)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Маркировка
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Применение watershed
img_watershed = img.copy()
markers = cv2.watershed(img_watershed, markers)
img_watershed[markers == -1] = [255, 0, 0]  # границы — синим

## 5. Визуализация

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 4, 1); plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)); plt.title('Оригинал'); plt.axis('off')
plt.subplot(2, 4, 2); plt.imshow(sobel_combined, cmap='gray'); plt.title('Sobel'); plt.axis('off')
plt.subplot(2, 4, 3); plt.imshow(canny_edges, cmap='gray'); plt.title('Canny'); plt.axis('off')
plt.subplot(2, 4, 4); plt.imshow(thresh_global, cmap='gray'); plt.title('Глобальный порог'); plt.axis('off')
plt.subplot(2, 4, 5); plt.imshow(thresh_adaptive, cmap='gray'); plt.title('Адаптивный'); plt.axis('off')
plt.subplot(2, 4, 6); plt.imshow(thresh_otsu, cmap='gray'); plt.title('Otsu'); plt.axis('off')
plt.subplot(2, 4, 7); plt.imshow(cv2.cvtColor(img_watershed, cv2.COLOR_BGR2RGB)); plt.title('Watershed'); plt.axis('off')

plt.tight_layout()
plt.show()

## Выводы

Какой метод лучше справляется с перекрывающимися объектами? В каких условиях Otsu даёт сбой?

## Контрольные вопросы:

1. В чём принцип работы оператора Canny? Какие этапы он включает?
2. Почему Sobel-оператор чувствителен к шуму? Как это компенсируется на практике?
3. Как работает метод Otsu? При каких условиях он даёт неудовлетворительные результаты?
4. В чём разница между глобальной и адаптивной пороговой сегментацией?
5. Почему метод водораздела (watershed) может приводить к пере-сегментации? Как этого избежать?
6. Как морфологические операции (эрозия, дилатация) помогают улучшить сегментацию?
7. Можно ли использовать watershed для сегментации перекрывающихся резервуаров на спутниковом снимке? Обоснуйте.
8. Как выбрать оптимальный размер ядра для морфологических операций?